In [2]:
import sys

from pathlib import Path
main_path = Path().absolute().parent
data_path = main_path / 'data'
setting_path = main_path / 'setting_files'

sys.path.append(str(main_path))

import torch
import yaml
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from src.nlu_models import NLUModel
from src.nlu_utils import NLUDataModule

with (setting_path / 'eval_settings.yml').open('r') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

data_module_settings = settings['data_module']
model_settings = settings['model']
trainer_settings = settings['trainer']

data_module = NLUDataModule(
    train_path=data_path / data_module_settings['train_file'], 
    valid_path=data_path / data_module_settings['valid_file'],
    test_path=data_path / data_module_settings['test_file'],
    labels_path=data_path / data_module_settings['labels_file'],
    batch_size=data_module_settings['batch_size'], 
    max_len=data_module_settings['max_len'],
    num_workers=data_module_settings['num_workers'],
    seed=settings['seed']
)


hparams = {
    'stage': model_settings['stage'],
    'model_path': model_settings['model_path'], 
    'intent_size': len(data_module.intents2id), 
    'tags_size': len(data_module.tags2id), 
    'lr': model_settings['lr'],
    'weight_decay_rate': model_settings['weight_decay_rate'],
    'loss_type': model_settings['loss_type'],
    'multigpu': True if trainer_settings['n_gpus'] > 1 else False
}
for k, v in model_settings['schedular'].items():
    hparams[f'schedular_{k}'] = v
if model_settings['loss_type'] == 'focal':
    for k, v in model_settings['focal'].items():
        hparams[f'focal_{k}'] = v

# model = NLUModel(**hparams)

log_path = main_path / 'logs'
checkpoint_path = str(main_path / 'checkpoints' / 'nlu_simple' / 'best_model.ckpt')

seed_everything(seed=settings['seed'])
trainer = pl.Trainer(
    gpus=trainer_settings['n_gpus'], 
    max_epochs=trainer_settings['n_epochs'], 
    num_sanity_val_steps=trainer_settings['num_sanity_val_steps'],
    deterministic=True,
)
checkpoint = torch.load(checkpoint_path)
model = NLUModel(**hparams)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

Global seed set to 777
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

NLUModel(
  (bert_ner): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [3]:
checkpoint['state_dict']['bert_ner.classifier.weight']

tensor([[ 0.1452,  0.0020,  0.0735,  ..., -0.1143,  0.0990,  0.0298],
        [-0.1532,  0.0517, -0.1016,  ...,  0.1262, -0.1057,  0.0031],
        [-0.1117, -0.0042, -0.1022,  ...,  0.1792, -0.0997, -0.0655],
        ...,
        [-0.1974,  0.0736, -0.0785,  ...,  0.1057, -0.1053,  0.0168],
        [-0.1741,  0.0499, -0.0926,  ...,  0.0827, -0.1118,  0.0002],
        [-0.1739,  0.0532, -0.0713,  ...,  0.1173, -0.1118,  0.0228]],
       device='cuda:0')

In [4]:
from src.nlu_utils import NLUTokenizer

tokenizer = NLUTokenizer()

In [5]:
def predict(model, tokenizer, text):
    bert_encodes = tokenizer(
        text, 
        add_special_tokens=True, 
        truncation=True, 
        max_length=256,
        return_tensors='pt'
    )
    output = model.predict(**bert_encodes)
    return output

In [12]:
model.to('cuda')
model.eval()

NLUModel(
  (bert_ner): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [14]:
data = {"text": "current assets"}
text = data['text']
bert_encodes = tokenizer(
    text, 
    add_special_tokens=True, 
    truncation=True, 
    max_length=256,
    return_tensors='pt'
).to('cuda')
# o = predict(model, tokenizer, text)
# o

In [15]:
o = model(**bert_encodes)

In [54]:
from collections import Counter
import json

a = Counter([1, 1, 3, 4])
json.dumps(a)

'{"1": 2, "3": 1, "4": 1}'

In [51]:
torch.manual_seed(4)
o = torch.rand(3, 3) * 10
p = torch.LongTensor([0, 2, 1])
cs = [2000000, 4, 2]
w = torch.FloatTensor([1 - (c/sum(cs)) for c in cs])
loss_w = nn.CrossEntropyLoss(weight=w)
loss = nn.CrossEntropyLoss()


tensor([3.0000e-06, 1.0000e+00, 1.0000e+00])
original 1.28769850730896
weighted 1.5840201377868652


In [61]:
alpha = 0.95
gamma = 3
ce_loss = loss(o, p)
pt = torch.exp(-ce_loss)
F_loss = alpha * (1-pt)**gamma * ce_loss

ce_loss_w = loss_w(o, p)
pt = torch.exp(-ce_loss_w)
F_loss_w = alpha * (1-pt)**gamma * ce_loss_w

print(w)
print(f'original {loss(o, p):.4f}, {F_loss:.4f}')
print(f'weighted {loss_w(o, p):.4f}, {F_loss_w:.4f}')

tensor([3.0000e-06, 1.0000e+00, 1.0000e+00])
original 1.2877, 0.4644
weighted 1.5840, 0.7557


In [50]:
F_loss

tensor(0.1688)

In [24]:
loss(o, p)

tensor(1.1188)

In [135]:
o['tags'].argmax(-1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [102]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix

In [106]:
all_accs = []
id2tags = {v: k for k, v in data_module.tags2id.items()}

for x in tqdm(test_loader, total=len(test_loader)):
    bert_inputs = {k: x[k] for k in ['input_ids', 'token_type_ids', 'attention_mask']}
    o = model.forward(**bert_inputs)
    y_true = x['tags'].view(-1).numpy()
    y_pred = o['tags'].argmax(-1).numpy()
    acc_tags = accuracy_score(y_true, y_pred)
    # m_tags = confusion_matrix(y_true, y_pred, labels=data_module.)
    break
    # all_accs.append(acc)

  0%|          | 0/47 [00:00<?, ?it/s]

{'tags': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'intent': tensor([0])}

In [17]:
bert_encodes = tokenizer(
    text, 
    add_special_tokens=True, 
    truncation=True, 
    max_length=256,
    return_tensors='pt'
)

In [10]:
import torch.nn as nn
ce = nn.CrossEntropyLoss(reduction='none')

In [37]:
logits = torch.FloatTensor([
    [9.3, -1.3, 0.14],
    [1.5, 7.3, 5.4],
    [5.3, -1.3, 0.14],
])
labels = torch.LongTensor([0, 0, 1])
print(logits.softmax(1).numpy().round(3))

[[1.    0.    0.   ]
 [0.003 0.868 0.13 ]
 [0.993 0.001 0.006]]


In [39]:
ce_loss = ce(logits, labels)
pt = torch.exp(-ce_loss)
gamma = 2
alpha = 0.75
loss = alpha * (1-pt)**gamma * ce_loss

print(ce_loss.numpy().round(3))
print(loss.numpy().round(3))

[0.    5.942 6.607]
[0.    4.433 4.942]


In [18]:
o = model(**bert_encodes)

In [19]:
o

{'tags': tensor([[ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0800, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0800, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0799, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0799, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.